In [ ]:
%matplotlib notebook
# Interactive backend required for model exploration
import gillespy2
from gillespy2.solvers.numpy import NumPySSASolver 
import numpy as np

### Define Gillespy2 model

In [ ]:
class ToggleSwitch(gillespy2.Model):
    """ Gardner et al. Nature (1999)
    'Construction of a genetic toggle switch in Escherichia coli'
    """
    def __init__(self, parameter_values=None):
        # Initialize the model.
        gillespy2.Model.__init__(self, name="toggle_switch")
        # Parameters
        alpha1 = gillespy2.Parameter(name='alpha1', expression=1)
        alpha2 = gillespy2.Parameter(name='alpha2', expression=1)
        beta = gillespy2.Parameter(name='beta', expression="2.0")
        gamma = gillespy2.Parameter(name='gamma', expression="2.0")
        mu = gillespy2.Parameter(name='mu', expression=1.0)
        self.add_parameter([alpha1, alpha2, beta, gamma, mu])

        # Species
        U = gillespy2.Species(name='U', initial_value=10)
        V = gillespy2.Species(name='V', initial_value=10)
        self.add_species([U, V])

        # Reactions
        cu = gillespy2.Reaction(name="r1",reactants={}, products={U:1},
                propensity_function="alpha1/(1+pow(V,beta))")
        cv = gillespy2.Reaction(name="r2",reactants={}, products={V:1},
                propensity_function="alpha2/(1+pow(U,gamma))")
        du = gillespy2.Reaction(name="r3",reactants={U:1}, products={},
                rate=mu)
        dv = gillespy2.Reaction(name="r4",reactants={V:1}, products={},
                rate=mu)
        self.add_reaction([cu,cv,du,dv])
        self.timespan(np.linspace(0,50,101))

toggle_model = ToggleSwitch()

### Use Sciope's Gillespy2 wrapper to extract simulator and parameters

In [ ]:
from sciope.utilities.gillespy2 import wrapper

settings = {"solver": NumPySSASolver, "number_of_trajectories":10, "show_labels":True}
simulator = wrapper.get_simulator(gillespy_model=toggle_model, run_settings=settings, species_of_interest=["U", "V"])

expression_array = wrapper.get_parameter_expression_array(toggle_model)

### Use LHD for sampling and TSFRESH minimal summary statistics
To generate a LHD for exploration we need to start a dask client. Points will be generated using distributed resources and persited at worker nodes,
we can then draw random samples from the LHD to local RAM. In this example we use a local cluster.

In [ ]:
from dask.distributed import Client
from sciope.designs import latin_hypercube_sampling
from sciope.utilities.summarystats.auto_tsfresh import SummariesTSFRESH

c = Client()
lhc = latin_hypercube_sampling.LatinHypercube(xmin=expression_array, xmax=expression_array*3)
lhc.generate_array(1000) #creates a LHD of size 1000

#will use default minimal set of features
summary_stats = SummariesTSFRESH()

### Start local cluster using dask client and Model exploration with StochMET

In [ ]:
from sciope.stochmet.stochmet import StochMET

met = StochMET(simulator, lhc, summary_stats)

### Run parameter sweep

In [ ]:
met.compute(n_points=500, chunk_size=10)

### Explore the result
Here we will explore parameter points expressed in feature space using a dimension reduction method. User can interact with points and label points according to different model behavior

In [ ]:
#First lets add some appropiate information about the model and features
met.data.configurations['listOfParameters'] = list(toggle_model.listOfParameters.keys())
met.data.configurations['listOfSpecies'] = list(toggle_model.listOfSpecies.keys())
met.data.configurations['listOfSummaries'] = met.summaries.features
met.data.configurations['timepoints'] = toggle_model.tspan

In [ ]:
# Here we use UMAP for dimension reduction
met.explore(dr_method='umap')

## Once a few points has been added we can use Semi-supervised learning

In [ ]:
from sciope.models.label_propagation import LPModel
#here lets use the dimension reduction embedding as input data
data = met.dr_model.embedding_

model_lp = LPModel()
#train using basinhopping
model_lp.train(data, met.data.user_labels, min_=0.01, max_=10, niter=50)

In [ ]:
# just to vislualize the result we will map the label distribution to the user_labels (will enable us to see the LP model 
# output when using method "explore")

user_labels = np.copy(met.data.user_labels)
#takes the label corresponding to index 0
met.data.user_labels = model_lp.model.label_distributions_[:,0]

In [ ]:
met.explore(dr_method='umap', from_distributed=False)

In [ ]:
met.data.user_labels = user_labels